# VLMモデル比較



### 共通のimport

In [ ]:
# 共通のimport
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import requests
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from PIL import Image, ImageDraw
import pickle
import torch
from transformers import BitsAndBytesConfig
from transformers import pipeline
from tqdm import tqdm
import openai
from openai import OpenAI
from src.utils import download_image_from_s3


import torch
from transformers import AutoTokenizer, AutoModelForVision2Seq, AutoImageProcessor
from transformers import AutoProcessor, Blip2ForConditionalGeneration

from PIL import Image
import requests

### 検証用の画像を準備

In [ ]:
df = pd.read_pickle('/media/sj-archimedes/data/share/OddAI_Library_practice/data08/fba_test_20231001-20231031.pkl')

In [ ]:
df = df.query('creative_type == "image"')
df = df[['creative_media_hash', 'creative_media_url', 'headline', 'description', 'genre']]
df['creative_media_hash'] = df['creative_media_hash'].map(lambda x: x[0])
df['creative_media_url'] = df['creative_media_url'].map(lambda x: x[0])
df['headline'] = df['headline'].map(lambda x: x[0])
df['description'] = df['description'].map(lambda x: x[0])
df = df.drop_duplicates('creative_media_hash')

In [ ]:
df['genre'].value_counts()

In [ ]:
target_genres = ['FMCG（日用消費財）', 'ヘルス&ビューティ', '人材', '教育・資格', 'ショッピング']

sample_image_dict = {}
for genre in target_genres:
    _df = df.query('genre == @genre')
    sample_image_dict[genre] = [download_image_from_s3(s3_url) for s3_url in _df['creative_media_url'].sample(3)]

In [ ]:
# with open('./sample_image/sample_image_dict.pkl', 'wb') as w:
#     pickle.dump(sample_image_dict, w)

with open('./sample_image/sample_image_dict.pkl', 'rb') as r:
    sample_image_dict = pickle.load(r)

In [ ]:
for genre, img in sample_image_dict.items():
    fig, ax = plt.subplots(1,3,figsize=(15,5))
    for i in range(3):
        ax[i].imshow(img[i])
    plt.show()

## stabilityai/japanese-stable-vlm

- 商用利用可で日本語対応のVLMモデル

In [ ]:
model_id = "/media/sj-archimedes/data/03_pretrained_model/llm/stabilityai/japanese-stable-vlm"
model_kwargs = {"trust_remote_code": True, "low_cpu_mem_usage": True}
model_kwargs["variant"] = "fp16"
model_kwargs["torch_dtype"] = torch.float16
model_kwargs["device_map"] = "auto"

model = AutoModelForVision2Seq.from_pretrained(model_id, **model_kwargs)
processor = AutoImageProcessor.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = model.eval()

In [ ]:
def stable_vlm_caption(image_list, prompt):
    fig, ax = plt.subplots(1,3,figsize=(15,5))
    for i in range(3):
        ax[i].imshow(image_list[i])
    plt.show()

    caption_result = []
    for img in image_list:
        # 入力の準備
        inputs = processor(images=img, return_tensors="pt")
        text_encoding = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")
        inputs.update(text_encoding)
        
        # 推論の実行
        outputs = model.generate(
            **inputs.to(device=model.device),
            do_sample=True,
            num_beams=1,
            max_new_tokens=512,
            min_length=1,
            repetition_penalty=1.5,
            pad_token_id=tokenizer.eos_token_id
        )
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        caption_result.append(generated_text)
    return caption_result

In [ ]:
prompt = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
画像を詳細に述べてください。

### 応答: 
"""

for genre, img in sample_image_dict.items():
    caption_list = stable_vlm_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- 単純な画像を説明させるようなキャプション生成は全然だめ
- 下記のようにVQAの使い方もできるようなので、質問形式にしてみる。

In [ ]:
prompt = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
与えられた画像を下に、質問に答えてください。
人はいますか？

### 応答: 
"""

for genre, img in sample_image_dict.items():
    caption_list = stable_vlm_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- 答えられたり、答えられなかったり。微妙
- yes/noで回答してくれたほうが後々便利なので、試す

In [ ]:
prompt = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
与えられた画像を下に、質問に答えてください。
人はいますか？はい/いいえ で答えてください。

### 応答: 
"""

for genre, img in sample_image_dict.items():
    caption_list = stable_vlm_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- はいかいいえで答えてくれ、という指示に従うことがそもそも難しそう。
- ただある程度回答はできている
- もう少し広告ならではの質問をしてみる。訴求観点を問う。

In [ ]:
prompt = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
与えられた画像を下に、質問に答えてください。
割引や特典を促す広告ですか？

### 応答: 
"""

for genre, img in sample_image_dict.items():
    caption_list = stable_vlm_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- ある程度答えられているような、でも精度は悪い
- もう少し定性的な観点を聞いてみる

In [ ]:
prompt = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
与えられた画像を下に、質問に答えてください。
この広告画像の魅力な点を教えてください。

### 応答: 
"""

for genre, img in sample_image_dict.items():
    caption_list = stable_vlm_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- たまにいい感じに答えられたりしているが、基本的に全然ダメ

## Salesforce/blip2-opt-2.7b

In [ ]:
model_id = '/media/sj-archimedes/data/03_pretrained_model/llm/salesforce/blip2-opt-2.7b'
processor = AutoProcessor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id, device_map='auto')

In [ ]:
def blip2_caption(image_list, prompt=None):
    fig, ax = plt.subplots(1,3,figsize=(15,5))
    for i in range(3):
        ax[i].imshow(image_list[i])
    plt.show()

    caption_result = []
    for img in image_list:
        # 入力の準備
        inputs = processor(img, text=prompt, return_tensors="pt").to(model.device)
        generated_ids = model.generate(**inputs, max_new_tokens=128)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        caption_result.append(generated_text)
    return caption_result

In [ ]:
for genre, img in sample_image_dict.items():
    caption_list = blip2_caption(img)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- ある程度写っているものを捉えていることはできている

In [ ]:
prompt = "Question: Is there anyone there? Answer:"

for genre, img in sample_image_dict.items():
    caption_list = blip2_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- 人がいるかどうかを尋ねる簡単な質問でも全然答えられていない。
- やはり広告画像をBLIP-2で扱うのは難しい、と思われる。

In [ ]:
prompt = "Question: Does it include content promoting discounts or benefits? Answer:"

for genre, img in sample_image_dict.items():
    caption_list = blip2_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

- すべてにYesと答えてしまった。ちょっとBLIP-2は難しそうだ

## GRiT

- https://zenn.dev/turing_motors/articles/ai-movie-searcher#grit
- こちらの記事で紹介されているように、detection & captioninigを同時に行うモデル

## DETR

In [ ]:
# 学習済FeatureExtractorをロード
from layout_detr.src.layout_detr import LayoutDetrFeatureExtractor
from layout_detr.src.layout_detr import LayoutDetrForObjectDetection
processor = LayoutDetrFeatureExtractor()
model = LayoutDetrForObjectDetection.from_pretrained('v1-nlabel-1')
model.to('cuda:0')
print()

In [ ]:
img = sample_image_dict['人材'][0]
img

In [ ]:
inputs = processor.preprocess(img, return_tensors='pt')
inputs = {k:v.to(model.device) for k, v in inputs.items()}
outputs = model(**inputs)

In [ ]:
target_size = torch.Tensor([img.size[::-1]])  # 縦横反転
postprocessed = processor.post_process_object_detection(outputs, target_sizes=target_size, threshold=0)[0]

In [ ]:
scores = postprocessed['scores'].cpu().detach().numpy()
boxes = postprocessed['boxes'].cpu().detach().numpy()

N_BOXES = 8
ids = scores.argsort()[::-1][:N_BOXES]  # (１つの式にスライスとストライドを同時に使わない. by Effective Python)
ids = ids[:6]
scores = scores[ids]
boxes = boxes[ids]

# drawer = ImageDraw.Draw(img)
# for box in boxes:
#     drawer.rectangle(box, outline='red', width=3)
# img

In [ ]:
for box in boxes:
    display(img.crop(box))

## LLaVA-1.5-7b
- 商用利用はできないけど、OSSでは最強クラスのモデル
- 日本語で出力してもらうようにプロンプトを書けば日本語で生成してくれるけど、英語で生成するときよりも性能下がるので、基本的に英語ベースでの使用を前提
- そのため、出力結果をGPT-3.5で翻訳する

In [ ]:
openai.api_key = os.environ['OPENAI_API_KEY']

model_id = "/media/sj-archimedes/data/03_pretrained_model/llm/llava-hf/llava-1.5-7b-hf"

model = pipeline(
    "image-to-text",
    model=model_id,
    device_map='auto',
)

In [ ]:
def llava_caption(image_list, prompt, chatgpt=True):
    fig, ax = plt.subplots(1,3,figsize=(15,5))
    for i in range(3):
        ax[i].imshow(image_list[i])
    plt.show()

    for img in image_list:
        outputs = model(
            img,
            prompt=prompt,
            generate_kwargs={"max_new_tokens": 512}
        )
        
        response = outputs[0]['generated_text'].split('\nASSISTANT: ')[1]
        
        print(response)

        if chatgpt:
            instruction = f"次の文章を日本語に翻訳してください。\n{response}"
            
            
            client = OpenAI()
            
            response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": instruction},
              ]
            )
            
            response = response.choices[0].message.content
            
            print(response)
        print('\n==========\n')

In [ ]:
# prompt = "USER: <image>\nPlease explain in detail how this advertisement image has been designed to appear attractive to consumers.\nASSISTANT:"
prompt = "USER: <image>\nPlease describe this image.\nASSISTANT:"
for genre, image_list in sample_image_dict.items():
    llava_caption(image_list, prompt)

- ちらほら間違いはあるけど、基本的な情報はちゃんと認識できている。
- 以下の例は広告画像の訴求内容やデザイン的な工夫を説明させるプロンプトを投げている
- 詳しく説明しろ、といったからなのか、単純に画像を説明してくれ、というときより、生成されるテキストが長くなる

In [ ]:
# この広告画像がどのようにして消費者に魅力的に見えるようにデザインされているか、詳しく説明してください。
prompt = "USER: <image>\nPlease explain in detail how this advertisement image has been designed to appear attractive to consumers.\nASSISTANT:"
# prompt = "USER: <image>\nPlease describe this image.\nASSISTANT:"
for genre, image_list in sample_image_dict.items():
    llava_caption(image_list, prompt)

- 結構いい感じに見える
- yes or no の使い方も一応やってみる

In [ ]:
prompt = "USER: <image>\nIs there a person in this image? Please answer yes or no.\nASSISTANT:"
for genre, image_list in sample_image_dict.items():
    llava_caption(image_list, prompt, chatgpt=False)

- 完璧
- 手だけ写っているものもYesと答えるけど、この辺はプロンプトで制御できそう
- 一度に複数のタグをyes noで答えられるかもやってみる

In [ ]:
# 画像をよく解析した上で、以下の質問にはいかいいえでこたえてください。
# 1. 人は写っていますか？
# 2. 屋外ですか？
# 3. 動物がいますか？
# 4. 中央にキャッチコピーが配置されていますか？
# 5. 画像が分割されたデザインになっていますか？
prompt = """USER: <image>\nAfter carefully analyzing the image, answer the following questions with a yes or no:
1. Is there a person in the picture?
2. Is it outdoors?
3. Are there any animals?
4. Is there a catchphrase placed in the center?
5. Is the image designed in a split layout?
ASSISTANT:"""
for genre, image_list in sample_image_dict.items():
    llava_caption(image_list, prompt, chatgpt=False)

- 複数同時に回答するのは無理らしい
- ただし、はいかいいえで回答する制限をなくしたら、ちゃんと箇条書きで答えてくれる

In [ ]:
# 画像をよく解析した上で、以下の質問に答えてください。
# 1. 人は写っていますか？
# 2. 屋外ですか？
# 3. 動物がいますか？
# 4. 中央にキャッチコピーが配置されていますか？
# 5. 画像が分割されたデザインになっていますか？
prompt = """USER: <image>\nAfter carefully analyzing the image, answer the following questions:
1. Is there a person in the picture?
2. Is it outdoors?
3. Are there any animals?
4. Is there a catchphrase placed in the center?
5. Is the image designed in a split layout?
ASSISTANT:"""
for genre, image_list in sample_image_dict.items():
    llava_caption(image_list, prompt, chatgpt=True)

- 箇条書きで回答はしてくれるようになった
- 答えられる質問とそうでない質問が明確になった
    - 広告画像のレイアウト的な質問（No.4,5は難しい）が、No.1,2,3のような一般的な写真に何が写っているかやその状況を問うような質問は得意そう

In [ ]:
prompt = "USER: <image>\nDoes it include content promoting discounts or benefits?\nASSISTANT:"
for genre, image_list in sample_image_dict.items():
    llava_caption(image_list, prompt, chatgpt=True)

- 以外にも全然ダメだった。Yesと答えやすいのかもしれない。

In [ ]:
>